<a href="https://colab.research.google.com/github/benevolent-machines/datamachine/blob/master/nb/mod_dpa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Benevolent Machines
### **DonorPerfect Analytics Bundle**
This read-only notebook serves as the public **dpanalytics** module that can be used by other notebooks as part of an Analytics Machine. It provides advanced analytics functions with information in a DonorPerfect database.    

#### **General Usage**
``` python
!pip install datamachine
import datamachine as dm
dpa = dm.import_notebook('dpanalytics')
```
After this code has run, the `npa` object should have the Class and function definitions that are contained in this notebook. You can then connect to your DonorPerfect instance with an API key by calling dpa.Data.connect():

``` python
DPO_API_KEY = '---your secret API key---'
dpa.Data.connect(DPO_API_KEY)
```
After you've connected to DonorPerfect, you can then invoke the methods in this notebook though that object.  For the following statement will show the weekly gift total.
``` python
dpa.weekly_gift_totals()
```
You can also invoke your own queries for data using the `dp.Data.query(sql)` call.  Here's an example:
``` python
df = dp.Data.query('''
    SELECT donor_id, gift_date, amount FROM dpgift
    WHERE gift_date > GETDATE()-(365*7)
''')
```

In [ ]:
#test
!pip install datamachine --quiet

In [ ]:
from IPython.display import Javascript
import datamachine as _nbm
import plotly.io as _pio
_pio.renderers.default='colab'
#display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

In [ ]:
class Data:
    import re
    import time
    from sys import getsizeof
    import requests as _requests # for calling the API
    from io import StringIO as _StringIO # string as file
    import xml.etree.cElementTree as _ET # parsing XML
    import pandas as _pd # target dataframe
    import json as _json
    import os as _os
    # test
    _api_url = 'https://www.donorperfect.net/prod/xmlrequest.asp'
    _api_key = ''
    custom = None
    myfile = 'dpanalytics.json'
    def connect(key='', quiet=False):
        if key == '': # then we use the file
            if Data._os.path.exists(Data.myfile):
                with open(Data.myfile) as f:
                    data_str = f.read()
                    data = Data._json.loads(data_str)
                    key = data['dpo_key']

        if key == '':
            print('Key is blank.  Either provide a valid key or {Data.myfile} file.')
            return

        Data._api_key = key
        df = Data.query("SELECT DB_NAME() db_name")
        db = df['db_name'].values[0]
        if quiet==False:
            print('Connected to ' + db)
        with open(Data.myfile, 'w') as f: # rewrite if needed to change key
            Data._json.dump({"dpo_key": key}, f, indent=4)

    def query(
        sql,
        chunk_trace=False,
        trace=False
    ):
        """Query the DonorPerfect API with SQL from XML to a pandas object

        A wrapper around the DonorPerfect API requests a SQL query, converts
        the XML response to a pandas object via a JSON object, and optionally
        collects a series of response chunks into a consolidated result set.

        Keyword Arguments
        ----------
        sql : str
            The SQL query to run against DonorPerfect.

            If ROW_NUMBER() OVER (ORDER BY col) AS rownum is used as the
            first column in the query, the function will issue as many
            requests as needed to chunk together a result set that can
            exceed the maximum number of rows that the API permits per
            request

        chunk_trace : boolean

            If enabled, chunking progress is shown to the user.

        trace : boolean

            If enable, the raw response of the first request is shown
            to the user and processing of the request is aborted.  This
            is mainly used to diagnose issues with the API.

        """
        if Data._api_key == '':
            print('query() requires an API key set first with connect()')
            return
        last_rownum = "0"
        chunked = False
        chunks = 0
        action = sql

        if 'ROW_NUMBER()' in sql[:80].upper():
            #print('chunked == True')
            # read as many rows as possible - clients range from 500 to 10,000
            action = "SELECT * FROM ( " + sql + " ) AS paged ORDER BY rownum"
            chunked = True
            if chunk_trace:
                print("   records chunks ")
                print("---------- ------ ")
                print("         0      0 ", end="") # dots to follow :)
        ja = [] # json array to join
        ctr = 1
        while True: # exit on empty chunk or if chunked is false
            ctr = ctr + 1
            if ctr > 20_000: # 10,000,000 records.. lol
                return # endless loop failsafe
            data = {"apikey": Data._requests.utils.unquote(Data._api_key), "action": action}
            api_response = Data._requests.post(url=Data._api_url, data=data)
            if trace:
                print(api_response.text.replace('\x11','Z'))
                return # get out

            if '<error>' in api_response.text:
                error_pattern = r"\[SQL Server\](.+)</error>"
                error_list = Data.re.findall(error_pattern, api_response.text)
                if len(error_list) >= 1:
                    chunk_trace and print('')
                    print('ERROR:', error_list[0])
                else:  # lets try another way
                    error_pattern = r"<error>(.+)</error>"
                    error_list = Data.re.findall(error_pattern, api_response.text)
                    if len(error_list) >= 1:
                        chunk_trace and print('')
                        print('ERROR:', error_list[0])
                    else:
                        print(api_response.text)
                return
            if api_response.text == "<result></result>":
                if chunks == 0:
                    chunk_trace and print('')
                    print('No results found')
                    return # no results
                break # end of chunking

            file = Data._StringIO(api_response.text.replace('\x11','')) # ugh ctrl-Q
            context = Data._ET.iterparse(file, events=("start","end"))
            context = iter(context)
            record = 1
            field = 1
            for event, elem in context:
                tag = elem.tag
                if (event == "start") and (tag =="result"):
                    ja.append('[')
                if (event == "start") and (tag == "record"):
                    if record > 1:
                        ja.append(',')
                    record = record + 1
                    field = 1
                    ja.append('{')
                if (event == "end") and (tag == "field"):
                    if field > 1:
                        ja.append(",")
                    field = field + 1
                    if elem.attrib['name'] == 'rownum':
                        last_rownum = elem.attrib['value']
                    ja.append("'" + elem.attrib['name']
                        + "': " + Data._json.dumps(elem.attrib['value']))
                if (event == "end") and (tag == "record"):
                    ja.append('}')
                if (event == "end") and (tag =="result"):
                    ja.append(']')
                value = elem.text
            if chunked == False:
                break
            chunks = chunks + 1
            if chunk_trace:
                print(".", end="")
                if chunks % 50 == 0:
                    print("\n{:10,d} {:6,d} ".format(
                        int(last_rownum), chunks), end="")
            #if chunks % 20 == 0:  # no need, really
            #    print('sleeping 1 second to throttle api use', chunks)
            #    time.sleep(1)
            action = ( "SELECT * FROM ( " +
                      sql + " ) AS paged WHERE rownum > " +
                      str(last_rownum) + " ORDER BY rownum" )
        if chunk_trace:
            print("\n{:10,d} {:6,d} ".format(int(last_rownum),  chunks))

        json_str = ''.join(ja)
        start = Data.time.process_time()
        json_str = json_str.replace('][',',') # fix chunked data

        json_file = Data._StringIO(json_str)
        df = Data._pd.DataFrame(eval(json_str))
        return df

    _activity_mappings = None
    def activity_mappings():
        import io
        import pandas as pd
        if Data._activity_mappings is None:
            Data._activity_mappings = pd.read_csv(io.StringIO(
"""depth,color,symbol,activity_code
1,brown,C,CC
1,brown,R,
1,brown,RS,RCPTSNT
2,orange,M,MA
2,orange,EO,EO
2,orange,VM,VM
2,orange,EI,EI
3,orange,LT,LT
4,orange,TE,TE
4,orange,n,NOTES
4,orange,IV,INVITATION
4,blue,VO,VOL
5,purple,VI,VI
5,purple,EV,EVENT
5,purple,PC,PC
5,purple,ME,ME
5,purple,GP,GP
5,purple,GR,GR
""")).fillna('')
        return Data._activity_mappings

In [ ]:
#test
Data.connect(quiet=True) # use a file - NEVER enter a key here
_custom_link = "https://colab.research.google.com/drive/1vkSGp7L-EzatLDXCJyCMlmG3Tm0XR2gy"
custom = _nbm.import_nb(_custom_link)

In [ ]:
def title():
    from IPython.core.display import HTML
    display(HTML("""
        <h1>Golden Harvest Food Bank</h1>
        <h2 style="font-weight: normal; ">Fundraising Machine for 01/21/2023</h2>
    """))
#title()

In [ ]:
def summary():
    from IPython.core.display import HTML
    display(HTML("""
    <style>
      td { text-align: right; }
      tzd:first-child { text-align: right; font-weight: bold; }
      tr.hdr td { background-color: #dddddd; font-weight: bold; }
    </style>
    <h2>Weekly Summary</h2>
    <p>Here's a look at last week by the numbers:</p>
    <div style="display: inline-block; vertical-align: top;">
      <table>
        <tr class="hdr"><td>GIFTS</td><td>WEEK</td><td>MTD</td><td>FYTD</td></tr>
        <tr><td><a href="#gift_totals">Total</a></td><td>$##,###</td><td>$#,###,###</td><td>$##,###,###</td></tr>
        <tr><td>Recurring</td><td>$##,###</td><td>$##,###</td><td>$##,###</td></tr>
        <tr><td>Largest</td><td>$##,###</td><td>$##,###</td><td>$##,###</td></tr>
        <tr><td>Average</td><td>$###</td><td>$###</td><td>$###</td></tr>
        <tr><td>Median</td><td>$###</td><td>$###</td><td>$###</td></tr>
        <tr><td>Count</td><td>###</td><td>###</td><td>###</td></tr>
        <tr class="hdr"><td>DONORS</td><td>WEEK</td><td>MTD</td><td>FYTD</td></tr>
        <tr><td>Gave</td><td>###</td><td>###</td><td>###</td></tr>
        <tr><td>Lapsed</td><td>###</td><td>###</td><td>###</td></tr>
        <tr><td>New</td><td>###</td><td>###</td><td>###</td></tr>
        <tr><td>Retained</td><td>###</td><td>###</td><td>###</td></tr>
        <tr><td>Reactivated</td><td>###</td><td>###</td><td>###</td></tr>
        <tr class="hdr"><td>ACTIVITIES</td><td>WEEK</td><td>MTD</td><td>FYTD</td></tr>
        <tr><td>Contacts</td><td>###</td><td>###</td><td>###</td></tr>
        <tr><td>Receipts</td><td>###</td><td>###</td><td>###</td></tr>
        <tr><td>Calls</td><td>###</td><td>###</td><td>###</td></tr>
        <tr><td>Active Solicitations</td><td>##</td><td>##</td><td>##</td></tr>
      </table>
    </div>
    <div style="display: inline-block;vertical-align: top;">
      <table>
        <tr class="hdr"><td colspan=2>KPIS</td></tr>
        <tr><td>Annual Gift Growth</td><td>#.#%</td></tr>
        <tr><td>Annual Donor Retention</td><td>#.#%</td></tr>
        <tr><td>Sustained Gift Growth</td><td>#.#%</td></tr>
        <tr><td>Sustained Donor Retention</td><td>#.#%</td></tr>

        <tr class="hdr"><td colspan=2>NEXT</td></tr>
        <tr><td>Donor Lapse Risks</td><td>###</td></tr>
      </table>
    </div>
    """))
#summary()

In [ ]:
def weekly_gift_totals():
    from IPython.core.display import HTML
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    import pandas as pd

    display(HTML("""<a name="gift_totals"></a>
    <h2>Weekly Gift Totals</h2>
    """))
    sql = """
        DECLARE @Date datetime=GETDATE() __ADJ__
        SELECT @Date = @Date - DATEPART(dw, @Date)
        SELECT
          DATEADD(DAY, 7-DATEPART(WEEKDAY, gift_date), CAST(gift_date AS DATE)) week_ending,
          SUM(amount) amt, COUNT(*) cnt
        FROM dpgift
        WHERE gift_date BETWEEN @Date - (13*7) AND @Date
        AND record_type = 'G'
        AND amount > 0
        GROUP BY DATEADD(DAY, 7-DATEPART(WEEKDAY, gift_date), CAST(gift_date AS DATE))
        ORDER BY week_ending
    """

    cydf = Data.query(sql.replace('__ADJ__',''))     # today
    cydf['week_ending'] = pd.to_datetime(cydf['week_ending'])
    cydf['cnt'] = pd.to_numeric(cydf['cnt'])
    cydf['amt'] = pd.to_numeric(cydf['amt'])

    pydf = Data.query(sql.replace('__ADJ__','-364')) # 52 weeks ago
    pydf['week_ending'] = pd.to_datetime(pydf['week_ending'])
    pydf['cnt'] = pd.to_numeric(pydf['cnt'])
    pydf['amt'] = pd.to_numeric(pydf['amt'])

    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.3,
        specs=[[{"secondary_y": True}], [{"secondary_y": True}]],
        subplot_titles=("Current Rolling Quarter", "Prior Year's Rolling Quarter"),
    )
    fig.update_layout(title="Weekly Gift Totals")
    fig.add_trace(go.Bar(x=cydf['week_ending'], y=cydf['amt'], marker_color='green',
                    name='Amount', opacity=0.5, legendgroup="1"), secondary_y=False, row=1, col=1)
    fig.add_trace(go.Scatter(x=cydf['week_ending'], y=cydf['cnt'], marker_color='black',
                    mode='lines+markers',
                    name='Count', legendgroup="1"), secondary_y=True, row=1, col=1)
    fig.add_trace(go.Bar(x=pydf['week_ending'], y=pydf['amt'], marker_color='green',
                    name='Amount', opacity=0.5, legendgroup = "2"), secondary_y=False, row=2, col=1)
    fig.add_trace(go.Scatter(x=pydf['week_ending'], y=pydf['cnt'], marker_color="black",
                    mode='lines+markers',
                    name='Count', legendgroup="2"), secondary_y=True, row=2, col=1)
    fig.update_layout(yaxis2=dict(tickformat=''))
    fig.update_layout(height=500, width=700) #, title_text="Side By Side Subplots")
    fig.update_layout(scene_yaxis = dict(exponentformat= "none", separatethousands= True), legend_tracegroupgap=180 )
    fig.update_layout(yaxis_tickformat = '$,', )
    fig.update_layout(yaxis2_tickformat = ',')
    fig.update_layout(yaxis3_tickformat = '$,', )
    fig.update_layout(yaxis4_tickformat = ',')
    fig.update_layout(yaxis_range = [0,800_000])
    fig.update_layout(yaxis2_range = [0,3_000])
    fig.update_layout(yaxis_title = 'Amount')
    fig.update_layout(yaxis3_title = 'Amount')
    fig.update_layout(xaxis=dict(tickformat="%m/%d/%y"), xaxis2=dict(tickformat="%m/%d/%y"))
    fig.update_xaxes(tickmode='linear', type='date')
    fig.update_xaxes(dtick=86400000*7, tick0="2021-10-23")
    fig.show()
#weekly_gift_totals()

In [ ]:
def weekly_gift_history():
    from IPython.core.display import HTML
    import plotly.express as px
    import pandas as pd
    display(HTML("""<a name="gift_totals2"></a>
    <h2>Weekly Gift Totals</h2>

    """))
    weekly_gifts = Data.query("""
        declare @Date datetime=GETDATE()-364
        select @Date = @Date - DATEPART(dw, @Date)
        SELECT
          DATEADD(DAY, 7-DATEPART(WEEKDAY, gift_date), CAST(gift_date AS DATE)) week_ending,
          SUM(amount) amt, COUNT(*) cnt
        FROM dpgift
        WHERE gift_date BETWEEN @Date - (13*7) AND @Date
        AND record_type = 'G'
        AND amount > 0
        GROUP BY DATEADD(DAY, 7-DATEPART(WEEKDAY, gift_date), CAST(gift_date AS DATE))
        ORDER BY week_ending DESC
    """)
    weekly_gifts['week_ending'] = pd.to_datetime(weekly_gifts['week_ending'])
    weekly_gifts['amt'] = pd.to_numeric(weekly_gifts['amt'])
    weekly_gifts['amount'] = weekly_gifts['amt']

    fig = px.bar(weekly_gifts, x='week_ending', y='amt',
                    labels={
                        "week_ending": "Week Ending (Saturday)",
                        "amt": "Total Amount"
                    },title='Recent'
    )
    fig.update_traces(marker_color='green')
    fig.update_layout(
        margin=dict(l=20, r=20, t=30, b=20),
        paper_bgcolor="#fefefe",
    )
    fig.update_layout(width=800, height=240, bargap=0.1)
    fig.update_xaxes(tickangle= -80)
    fig.update_layout(xaxis=dict(tickformat="%Y-%m-%d"))
    fig.update_xaxes(tickmode='linear', type='date')
    fig.update_xaxes(dtick=86400000*7, tick0="2021-10-23")
    fig.show()
    display(weekly_gifts)

In [ ]:
zzz = '''Data.query("""
    SELECT ROW_NUMBER() OVER (ORDER BY g.gift_id) AS rownum,
        g.gift_date, g.amount, g.solicit_code, g.gl_code, g.created_date, g.created_by,
        d.donor_id,
        du.recency_segment, du.frequency_segment, du.monetary_segment, du.type_segment
    FROM dpgift g
    JOIN dp d ON d.donor_id = g.donor_id
    JOIN dpudf du ON du.donor_id = d.donor_id
    WHERE 1=1
    AND gift_date BETWEEN GETDATE()-DATEPART(dw, GETDATE()) - (1*7)
                      AND GETDATE()-DATEPART(dw, GETDATE())
    AND record_type = 'G'
    AND amount > 0
""")'''

In [ ]:
def weekly_gifts_pivot():
    from IPython.core.display import HTML
    display(HTML("""<a name="gift_pivot"></a>
    <h2>Weekly Gifts Pivot</h2>
    """))
    gdf = Data.query("""
        SELECT ROW_NUMBER() OVER (ORDER BY g.gift_id) AS rownum,
            g.gift_date, g.amount, g.solicit_code, g.gl_code, g.created_date, g.created_by,
            d.donor_id, d.state, d.city, d.zip, d.donor_type,
            du.recency_segment, du.frequency_segment, du.monetary_segment, du.type_segment
        FROM dpgift g
        JOIN dp d ON d.donor_id = g.donor_id
        JOIN dpudf du ON du.donor_id = d.donor_id
        WHERE 1=1
        AND gift_date BETWEEN GETDATE()-DATEPART(dw, GETDATE()) - (1*7)
                          AND GETDATE()-DATEPART(dw, GETDATE())
        AND record_type = 'G'
        AND amount > 0
    """)
    TEMPLATE = u"""
    <!DOCTYPE html>
    <html>
        <head>
            <meta charset="UTF-8">
            <title>PivotTable.js</title>
            <!-- external libs from cdnjs -->
            <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.11/c3.min.css">
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.11/c3.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/1.11.2/jquery.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.11.4/jquery-ui.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery-csv/0.71/jquery.csv-0.71.min.js"></script>
            <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/pivot.min.css">
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/pivot.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/d3_renderers.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/c3_renderers.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/export_renderers.min.js"></script>
            <style>
                body {font-family: Verdana;}
                .node {
                  border: solid 1px white;
                  font: 10px sans-serif;
                  line-height: 12px;
                  overflow: hidden;
                  position: absolute;
                  text-indent: 2px;
                }
                .c3-line, .c3-focused {stroke-width: 3px !important;}
                .c3-bar {stroke: white !important; stroke-width: 1;}
                .c3 text { font-size: 12px; color: grey;}
                .tick line {stroke: white;}
                .c3-axis path {stroke: grey;}
                .c3-circle { opacity: 1 !important; }
                .c3-xgrid-focus {visibility: hidden !important;}
            </style>
        </head>
        <body>
            <script type="text/javascript">
                $(function(){
                    $("#output").pivotUI(
                        $.csv.toArrays($("#output").text()),
                        $.extend({
                            renderers: $.extend(
                                $.pivotUtilities.renderers,
                                $.pivotUtilities.c3_renderers,
                                $.pivotUtilities.d3_renderers,
                                $.pivotUtilities.export_renderers
                                ),
                            hiddenAttributes: [""]
                        }, %(kwargs)s)
                    ).show();
                });
            </script>
            <div id="output" style="display: none;">%(csv)s</div>
        </body>
    </html>
    """

    import json, io
    def pivot_ui2(df, width="100%", height="500", **kwargs):
        csv = df.to_csv(encoding='utf8')
        df.to_csv('weekly_gifts.csv')
        if hasattr(csv, 'decode'):
            csv = csv.decode('utf8')
        #print(csv)
        srcdoc = TEMPLATE % dict(csv=csv, kwargs=json.dumps(kwargs))
        srcdoc = srcdoc.replace('"','&quot;')  # encode quotes
        display(HTML(('''
          <iframe
              width="100%"
              height="500"
              srcdoc="''' + srcdoc + '''"
              frameborder="0"
              allowfullscreen
          ></iframe>
        ''')))
    pivot_ui2(gdf)
#weekly_gifts_pivot()

In [ ]:
def segmentation_pivot():
    from IPython.core.display import HTML
    display(HTML("""<a name="gift_totals"></a>
    <h2>Segmentation Pivot</h2>
    """))
    ddf = Data.query("""
        SELECT ROW_NUMBER() OVER (ORDER BY du.donor_id) AS rownum,
          recency_segment, frequency_segment, monetary_segment, type_segment
        FROM dpudf du
    """)
    TEMPLATE = u"""
    <!DOCTYPE html>
    <html>
        <head>
            <meta charset="UTF-8">
            <title>PivotTable.js</title>
            <!-- external libs from cdnjs -->
            <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.11/c3.min.css">
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.11/c3.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/1.11.2/jquery.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.11.4/jquery-ui.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery-csv/0.71/jquery.csv-0.71.min.js"></script>
            <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/pivot.min.css">
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/pivot.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/d3_renderers.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/c3_renderers.min.js"></script>
            <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/export_renderers.min.js"></script>
            <style>
                body {font-family: Verdana;}
                .node {
                  border: solid 1px white;
                  font: 10px sans-serif;
                  line-height: 12px;
                  overflow: hidden;
                  position: absolute;
                  text-indent: 2px;
                }
                .c3-line, .c3-focused {stroke-width: 3px !important;}
                .c3-bar {stroke: white !important; stroke-width: 1;}
                .c3 text { font-size: 12px; color: grey;}
                .tick line {stroke: white;}
                .c3-axis path {stroke: grey;}
                .c3-circle { opacity: 1 !important; }
                .c3-xgrid-focus {visibility: hidden !important;}
            </style>
        </head>
        <body>
            <script type="text/javascript">
                $(function(){
                    $("#output").pivotUI(
                        $.csv.toArrays($("#output").text()),
                        $.extend({
                            renderers: $.extend(
                                $.pivotUtilities.renderers,
                                $.pivotUtilities.c3_renderers,
                                $.pivotUtilities.d3_renderers,
                                $.pivotUtilities.export_renderers
                                ),
                            hiddenAttributes: [""]
                        }, %(kwargs)s)
                    ).show();
                });
            </script>
            <div id="output" style="display: none;">%(csv)s</div>
        </body>
    </html>
    """

    import json, io
    def pivot_ui2(df, width="100%", height="500", **kwargs):
        csv = df.to_csv(encoding='utf8')
        if hasattr(csv, 'decode'):
            csv = csv.decode('utf8')
        #print(csv)
        srcdoc = TEMPLATE % dict(csv=csv, kwargs=json.dumps(kwargs))
        srcdoc = srcdoc.replace('"','&quot;')  # encode quotes
        display(HTML(('''
          <iframe
              width="100%"
              height="500"
              srcdoc="''' + srcdoc + '''"
              frameborder="0"
              allowfullscreen
          ></iframe>
        ''')))
    pivot_ui2(ddf)
#segmentation_pivot()

In [ ]:
def solicitations():
    from IPython.core.display import HTML
    import numpy as np
    import pandas as pd
    display(HTML("""<a name="gift_totals"></a>
    <h2>Solicitations</h2>
    """))
    sdf = Data.query("""
        SELECT ROW_NUMBER() OVER (ORDER BY x.week_ending DESC, SUM(cnt) DESC) AS rownum,
          week_ending, solicit_code, SUM(cnt) cnt, SUM(amount) amount
        FROM (
          SELECT
            DATEADD(DAY, 8-DATEPART(WEEKDAY, gift_date), CAST(gift_date AS DATE)) week_ending,
            solicit_code,
            1 cnt, amount
          FROM dpgift
          WHERE gift_date >= '1/1/2022'
          AND record_type = 'G'
          AND amount > 0
        ) AS x
        GROUP BY week_ending, solicit_code
    """)
    sdf['cnt'] = pd.to_numeric(sdf['cnt'])
    sdf['amount'] = pd.to_numeric(sdf['amount'])
    sdf['week_ending'] = pd.to_datetime(sdf['week_ending'])
    sdf['solicit_code'] = np.where(
        sdf['solicit_code'].str.startswith('ADM'),
        sdf['solicit_code'].str[:6], sdf['solicit_code'])
    sdf['solicit_code'] = np.where(
        sdf['solicit_code'].str.startswith('adm'),
        sdf['solicit_code'].str[:6], sdf['solicit_code'])
    sdf['solicit_code'] = np.where(
        sdf['solicit_code'].str.startswith('DDM'),
        sdf['solicit_code'].str[:6], sdf['solicit_code'])
    sdf['solicit_code'] = np.where(
        sdf['solicit_code'].str.startswith('SPK'),
        sdf['solicit_code'].str[:6], sdf['solicit_code'])
    for y in ['21','22','23']:
        sdf['solicit_code'] = sdf['solicit_code'].str.replace(y,'yy')

    import plotly.express as px
    fig = px.bar(sdf, #[sdf['solicit_code'].isin(['MONTHLYxGIVING','TY'])],
                x="week_ending", y="cnt", color="solicit_code", title="Gift Counts by Solicitation over Weeks")
    fig.update_xaxes(tickformat="%b %d\n%Y")
    fig.update_yaxes()
    fig.show()

    fig = px.bar(sdf, #[sdf['solicit_code'].isin(['MONTHLYxGIVING','TY'])],
                x="week_ending", y="amount", color="solicit_code", title="Gift Amounts by Solicitation over Weeks")
    fig.update_xaxes(tickformat="%b %d\n%Y")
    fig.update_yaxes()
    fig.show()
#solicitations()

In [ ]:
def donor_timeline(donor_id):
    import pandas as pd
    import plotly.graph_objects as go
    import numpy as np
    from datetime import datetime, timedelta

    if 'custom' in globals() and hasattr(custom,'Data') and hasattr(custom.Data,'activity_mappings'):
        activity_mappings = custom.Data.activity_mappings()
    else:
        activity_mappings = Data.activity_mappings()
    #print(activity_mappings)

    ddf = Data.query('''
        SELECT donor_id, first_name, last_name, address, city, state, zip
        FROM dp
        WHERE donor_id = ''' + str(donor_id) + '''
    ''')
    donor = ddf.iloc[0] # first row

    gdf = Data.query('''
        SELECT gift_id, gift_date, amount, record_type, solicit_code
        FROM dpgift
        WHERE donor_id = ''' + str(donor_id) + '''
        --AND gift_date >= GETDATE()-(365*10)
        AND record_type = 'G'
        AND amount > 0
        ORDER BY 2
    ''')
    if gdf is None or gdf.empty:
        pass # return
    gdf['gift_date'] = pd.to_datetime(gdf['gift_date']) + pd.to_timedelta('12 hours')
    gdf['amount'] = pd.to_numeric(gdf['amount'])
    gdf['color'] = np.where(gdf['record_type'] == 'G', 'limegreen', 'blue')
    gdf['width'] = 9.0

    cdf = Data.query('''
        SELECT contact_id, contact_date, activity_code
        FROM dpcontact
        WHERE donor_id = ''' + str(donor_id) + '''
        --AND contact_date >= GETDATE()-(365*10)
        ORDER BY 2
    ''')

    if cdf is None or cdf.empty:
        pass #return
    cdf['contact_date'] = pd.to_datetime(cdf['contact_date']) + pd.to_timedelta('12 hours')
    cdf['width'] = 9.0
    cdf = cdf.merge(activity_mappings,on='activity_code')
    cdf['marker_size'] = 10 + cdf['depth']
    cdf['amount'] = gdf['amount'].max() * -0.2 * cdf['depth']

    extend = 10
    gift_hover = "<br><b>Gift Date</b>: %{x}<br><b>Amount</b>: $%{y:,.2f}<br><b>Record Type</b>: %{customdata}<extra></extra>"
    contact_hover = "<br><b>Contact Date</b>: %{x}<br><b>Activity</b>: %{customdata}<extra></extra>"

    min_date = gdf['gift_date'].min().date()
    max_date = (datetime.now() + timedelta(days=extend)).date()

    fig = go.Figure()
    fig.add_hline(y=0)
    years = [y for y in range(min_date.year + 1, max_date.year + 1)]
    for year in years:
        dt = datetime(year,1,1)
        fig.add_vline(x=dt, line_width=1, line_color="lightgrey", layer="below")

    fig.add_trace(
        go.Bar(name='Gifts',
              x=gdf['gift_date'],
              y=gdf['amount'],
              customdata=gdf['record_type'],
              hovertemplate=gift_hover,
              legendgroup='1',
              marker_color=gdf['color'], width=86400000*1))
    fig.add_trace(
        go.Scatter(name='Gifts',
              x=gdf['gift_date'],
              y=gdf['amount'],
              customdata=gdf['record_type'],
              hovertemplate=gift_hover,
              mode='markers',
              marker=dict(color='limegreen',size=10),
              legendgroup='1',
              showlegend=False))

    fig.add_trace(
        go.Bar(name='Contacts',
              x=cdf['contact_date'],
              y=cdf['amount'],
              customdata=cdf['activity_code'],
              hovertemplate=contact_hover,
              marker_color=cdf['color'], width=86400000*1))
    fig.add_trace(
        go.Scatter(name='Contacts',
              x=cdf['contact_date'],
              y=cdf['amount'],
              customdata=cdf['activity_code'],
              hovertemplate=contact_hover,
              mode='markers',
              marker=dict(color=cdf['color'],size=10 + cdf['depth']),
              showlegend=False))
    fig.update_layout(yaxis_title='Activity')
    fig.update_layout(barmode='relative')
    fig.update_layout(hovermode="closest")
    fig.update_layout(xaxis=dict(
        type="date",
        range= [max_date - timedelta(days=2*365), max_date],
        rangeselector=dict(
            buttons=list([
                dict(label="5y", count=5, step="year", stepmode="backward"),
                dict(label="2y", count=2, step="year", stepmode="backward"),
                dict(label="1y", count=365+7, step="day", stepmode="backward"),
                dict(label="6m", count=(365//2)+7, step="day", stepmode="backward"),
                dict(label="1m", count=38, step="day", stepmode="backward"),
                dict(label="14d", count=21, step="day", stepmode="backward")
            ]),
            x = 0,
            xanchor = 'left'
        ),
        rangeslider=dict(
            visible=True,  range=[min_date, max_date]
        )
    ))
    max = int(gdf['amount'].max())
    qin = max//5
    stp = round(qin,1-len(str(qin)))
    tickvals = [stp, stp*2, stp*3, stp*4, stp*5]
    fig.update_layout(
        yaxis=dict(
            tickformat="$,.0f",
            tickmode='array',
            tickvals=tickvals,
        )
    )
    #print('_'+donor['first_name']+'_')
    if donor['first_name'] is None or donor['first_name'] == '':
        donor_name = donor['last_name']
    else:
        donor_name = donor['first_name'] + ' ' + donor['last_name']

    donor_location = donor['address'] + ' ' + donor['city'] + ', ' + donor['state'] + '  ' + donor['zip']

    fig.update_layout(
        annotations=[
            dict(text=f"<b>{donor_name}</b> (<a target='_blank' href='http://www.donorperfect.net'>{str(donor['donor_id'])}</a>)",
                 x=0, xref="paper", y=1.29, yref="paper", showarrow=False, font=dict(size=14)),
            dict(text=f"{donor_location}",
                 x=0, xref="paper", y=1.21, yref="paper", showarrow=False, font=dict(size=14))

        ]
    )
    fig.show()
#donor_timeline(29512)

In [ ]:
#cdf

In [ ]:
def top_giver_timelines():
    # top 5 givers from last week
    dtg = Data.query('''
        SELECT TOP 5 donor_id, gift_date, amount
          FROM dpgift WHERE gift_date > GETDATE() - 7
        ORDER BY amount DESC
    ''')
    for donor_id in dtg['donor_id'].tolist():
        #for donor_id in range(29512,29513,1):
        donor_timeline(donor_id)
#top_giver_timelines()

In [ ]:
def top_lapser_timelines():
    # top 5 lapsers from last week
    dtg = Data.query('''
        SELECT TOP 10 g.donor_id, g.gift_date, g.amount
          FROM dpgift g
          JOIN dp d ON d.donor_id = g.donor_id
          WHERE g.gift_date BETWEEN GETDATE()-364-7 AND GETDATE()-364
            AND d.last_contrib_date <= g.gift_date
        ORDER BY g.amount DESC
    ''')
    for donor_id in dtg['donor_id'].tolist():
        #for donor_id in range(29512,29513,1):
        donor_timeline(donor_id)
#top_lapser_timelines()

In [ ]:
# called by the standard FAM notebooks
def demo():  # https://colab.research.google.com/drive/1wooLbiPbFJsAgKcLbR7A98zAclo4Q4Ks
    from IPython.display import Javascript
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 25000})'))
    #weekly_gift_totals()
    import nbmachine as nbm
    npa = nbm.load('npanalytics')
    npa.choropleth_by_state()
    if len(Data._api_key) > 0:
        weekly_gift_totals()